<div align="center">

# RIO Airbnb Project - Text Data Analysis
**Latest Update:** *27th February 2023*

</div>

---

#### **Import required libraries & dataset**

In [45]:
import sys, os, re
sys.path.append(os.path.abspath("../"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import metrics

import airbnblib.cleaning as cln

In [46]:
# Gets the latest cleaned version of the dataset
newest_listing_zip = os.listdir("../dataset_zipped/listings/")[-1]
listings_df = pd.read_csv(f"../dataset_zipped/listings/{newest_listing_zip}")

In [47]:
listings_df.head()

,Unnamed: 0,id,name,description,neighborhood_overview,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,...,hair_dryer,hot_water,long_term_stays_allowed,smoke_alarm,refrigerator,kitchen,air_conditioning,tv,essentials,zip_code
0,0,183319.0,Panoramic Ocean View Venice Beach,Craftsmen style penthouse with ocean view from...,About as close to the beach as you can live in...,-15180,Serious yoga practice. Been studying for 25 y...,within a day,100%,100%,...,0,1,1,1,1,1,0,1,0,90291.0
1,1,51307.0,Spanish Bungalow Guest House LA CA. 30 plus ni...,PRIVATE GUEST HOUSE<br /><br /><b>The space</b...,"Local LA Community , shops and restaurants in...",-14867,"male , educated . Healthcare Professional\r\nC...",does not respond,NaN,NaN,...,0,1,1,1,1,1,1,1,1,90039.0
2,2,184314.0,Boho Chic Flat..Steps to Beach!,"Bright, airy, quiet 1 bdr located just steps f...",You are in one of the beach locations in Los A...,-15183,Health-Wellness Professional. Author. Passiona...,does not respond,NaN,NaN,...,1,0,1,1,0,1,0,1,1,90292.0
3,3,51498.0,Guest House With Its Own Entrance/Exit and Hot...,"Fully self-contained, separate structure, with...",We are close to Venice without the hassle of n...,-14868,Easy going hostess!! Enjoy your stay. \n\nI am...,within an hour,100%,88%,...,1,1,1,1,0,1,1,1,1,90066.0
4,4,109.0,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...",NaN,-14057,"Paolo Privitera, CEO Evensi\n\nPaolo, MIT MBA ...",does not respond,NaN,NaN,...,1,0,1,1,0,1,1,1,1,90230.0


In [ ]:
from bs4 import BeautifulSoup

def remove_html(text):
  return BeautifulSoup(text).get_text(separator=' ') if (type(text) == str) else ''

listings_df[['name', 'description', 'neighborhood_overview']].apply(remove_html)

In [50]:
from bs4 import BeautifulSoup

for col in ['name', 'description', 'neighborhood_overview']:
  listings_df[col] = [
    BeautifulSoup(html).get_text(separator=' ') if (type(html) != float) else ''
    for html in listings_df[col] 
  ]

C:\Users\Tran Minh Khoi\AppData\Local\Temp\ipykernel_12612\765176077.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  BeautifulSoup(html).get_text(separator=' ') if (type(html) != float) else ''
C:\Users\Tran Minh Khoi\AppData\Local\Temp\ipykernel_12612\765176077.py:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  BeautifulSoup(html).get_text(separator=' ') if (type(html) != float) else ''


In [51]:
listings_df.head()

,Unnamed: 0,id,name,description,neighborhood_overview,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,...,hair_dryer,hot_water,long_term_stays_allowed,smoke_alarm,refrigerator,kitchen,air_conditioning,tv,essentials,zip_code
0,0,183319.0,Panoramic Ocean View Venice Beach,Craftsmen style penthouse with ocean view from...,About as close to the beach as you can live in...,-15180,Serious yoga practice. Been studying for 25 y...,within a day,100%,100%,...,0,1,1,1,1,1,0,1,0,90291.0
1,1,51307.0,Spanish Bungalow Guest House LA CA. 30 plus ni...,"PRIVATE GUEST HOUSE The space Private, Guest h...","Local LA Community , shops and restaurants in...",-14867,"male , educated . Healthcare Professional\r\nC...",does not respond,NaN,NaN,...,0,1,1,1,1,1,1,1,1,90039.0
2,2,184314.0,Boho Chic Flat..Steps to Beach!,"Bright, airy, quiet 1 bdr located just steps f...",You are in one of the beach locations in Los A...,-15183,Health-Wellness Professional. Author. Passiona...,does not respond,NaN,NaN,...,1,0,1,1,0,1,0,1,1,90292.0
3,3,51498.0,Guest House With Its Own Entrance/Exit and Hot...,"Fully self-contained, separate structure, with...",We are close to Venice without the hassle of n...,-14868,Easy going hostess!! Enjoy your stay. \n\nI am...,within an hour,100%,88%,...,1,1,1,1,0,1,1,1,1,90066.0
4,4,109.0,Amazing bright elegant condo park front *UPGRA...,"*** Unit upgraded with new bamboo flooring, br...",,-14057,"Paolo Privitera, CEO Evensi\n\nPaolo, MIT MBA ...",does not respond,NaN,NaN,...,1,0,1,1,0,1,1,1,1,90230.0


In [54]:
from datetime import datetime

today = str(datetime.now()).split(" ")[0].replace('-', '')
listings_df.to_csv(f"../dataset/listings/listings_cleaned_{today}.csv", index=False)